In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import numpy as np

In [2]:
#client = Client()

In [3]:
client

Client Scheduler: tcp://127.0.0.1:32957 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.79 GB


In [ ]:
client.restart()

In [4]:
%%time

dtypes = {
    'reviewerID': np.str,
    'asin': np.str,
    'reviewerName': np.str,
    'helpful': np.object,
    'reviewTest': np.str,
    'overall': np.float64,
    'summary': np.str,
    'unixReviewTime': np.float64,
    'reviewTime': np.str
}

df = dd.read_csv('user_reviews.csv', dtype=dtypes)


CPU times: user 34 ms, sys: 7.27 ms, total: 41.3 ms
Wall time: 37.9 ms


In [6]:
%%time

#df_1 = df.groupby('asin')['overall'].count(split_out=4).reset_index()
df.head(5)

CPU times: user 50.9 ms, sys: 21.7 ms, total: 72.6 ms
Wall time: 925 ms


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2T0RJ91B0PQ03,B0016CRVLW,Gerald DeWitt,"[0, 0]",Beware! This is NOT the original single versi...,1.0,Poor Quality Alternate Take,1.400630e+09,"05 21, 2014"
1,A3TYW0XA8HSGWB,B00EKR5S0Q,Linda E. Larson,"[0, 0]",This is my new most favorite k-cup coffee. I c...,5.0,Vanilla Starbucks K-cups,1.398557e+09,"04 27, 2014"
2,A2CME0TQU2IVVB,B001AUPJVO,L5Momma,"[1, 1]",This headset is great! It worked in our 2007 ...,5.0,Awesome!,1.355875e+09,"12 19, 2012"
3,A2E5IDLX7R388S,B000055Y57,Jeff Andersen,"[0, 0]",Scofield is one of my favorite musicians and i...,5.0,Straight ahead Jazz with the Scofield twist,1.402358e+09,"06 10, 2014"
4,A3CIEMYUGV6ZMR,0545265355,Adroit,"[0, 0]","Wonderful book! I cried, well teared up at a f...",5.0,Greatest Book Ever!!!,1.334102e+09,"04 11, 2012"


In [47]:
%%time

# Subset of df
test_df = df[['reviewerID', 'overall', 'reviewTime', 'helpful']]
test_df = test_df.loc[:20000]
test_df.compute()

CPU times: user 27.7 s, sys: 4 s, total: 31.7 s
Wall time: 4min 41s


,reviewerID,overall,reviewTime,helpful
0,A2T0RJ91B0PQ03,1.0,"05 21, 2014","[0, 0]"
1,A3TYW0XA8HSGWB,5.0,"04 27, 2014","[0, 0]"
2,A2CME0TQU2IVVB,5.0,"12 19, 2012","[1, 1]"
3,A2E5IDLX7R388S,5.0,"06 10, 2014","[0, 0]"
4,A3CIEMYUGV6ZMR,5.0,"04 11, 2012","[0, 0]"
...,...,...,...,...
19996,AXERWVRMQL07S,3.0,"06 21, 2013","[0, 0]"
19997,AJWUV8T0WOXYO,4.0,"11 16, 2012","[0, 1]"
19998,A3W1MA9ELA00W2,5.0,"12 2, 2013","[0, 0]"
19999,A29ON7RAVEIU4P,3.0,"10 13, 2002","[0, 2]"


In [ ]:
%%time

# year of review
# Try using unixReviewTime, might be faster
#test_df['reviewYear'] = test_df['reviewTime'].apply(lambda x: int(x[-4:]), meta=np.int)
test_df['reviewYear'] = test_df['reviewTime'].apply(lambda x: int(x[-4:]), meta=np.int).compute()

In [ ]:
%%time
#instead of apply, use map_partitions

# string list to list
test_df['helpful votes'] = test_df['helpful'].apply(lambda x: x.strip('][').split(', ')[0], meta=np.int)

In [48]:
%%time

test_df['total votes'] = test_df['helpful'].apply(lambda x: x.strip('][').split(', ')[1], meta=np.int)



test_df.head(10)

AttributeError: 'Series' object has no attribute 'map_partition'

In [37]:
%%time

number_products_rated = test_df.groupby('reviewerID').count(split_out=4)

KeyboardInterrupt: 

In [28]:
%%time

avg_ratings = test_df.groupby('reviewerID')['overall'].mean(split_out=4)
number_products_rated.head(5)

CPU times: user 31.6 s, sys: 3.15 s, total: 34.7 s
Wall time: 6min 3s


,overall,reviewTime,helpful,reviewYear,helpful votes,total votes
reviewerID,,,,,,
A00010181745VTMHSO9TO,1,1,1,1,1,1
A00254261GWP2LJ0A209G,7,7,7,7,7,7
A00340661HI1IH02R8XZD,4,4,4,4,4,4
A00853013B8T3EPOGJTHN,1,1,1,1,1,1
A0091239FUEEOGVC8IVV,1,1,1,1,1,1


In [15]:
%%time

reviewing_since = test_df.groupby('reviewerID')['reviewYear'].min(split_out=4)

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 14.5 ms


In [16]:
%%time

helpful_votes = test_df.groupby('reviewerID')['helpful votes'].sum(split_out=4)

CPU times: user 9.91 ms, sys: 0 ns, total: 9.91 ms
Wall time: 9.39 ms


In [17]:
%%time
total_votes = test_df.groupby('reviewerID')['total votes'].sum(split_out=4)

CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 9.39 ms


In [18]:
%%time
reviewerID = number_products_rated.index

CPU times: user 777 µs, sys: 0 ns, total: 777 µs
Wall time: 787 µs


In [ ]:
%%time
result_df = dd.concat([number_products_rated, avg_ratings, reviewing_since, helpful_votes, total_votes], axis=1,
                      ignore_unknown_divisions=True)
result_df.head(10)

In [20]:
result_df.columns = ['number_products_rated', 'avg_ratings', 'reviewing_since', 'helpful_votes', 'total_votes']


In [21]:
result_df = result_df.reset_index()

In [22]:
%% time
result_df.head(10)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting


KilledWorker: ("('read-csv-assign-15e9aacc986cf71fb3842b45e3903613', 307)", <Worker 'tcp://127.0.0.1:34949', name: 0, memory: 0, processing: 336>)